# Analysis of the dataset

Here I have to check if the data set has null values and those things.

In [471]:
import pandas as pd
import numpy as np
import numpy.linalg as npl

df = pd.read_csv("IRIS_Training.csv", sep=";")
df


,sepal_length,sepal_width,petal_length,petal_width,species
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
85,6.7,3.1,4.7,1.5,Iris-versicolor
86,6.3,2.3,4.4,1.3,Iris-versicolor
87,5.6,3.0,4.1,1.3,Iris-versicolor
88,5.5,2.5,4.0,1.3,Iris-versicolor


In [472]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  90 non-null     float64
 1   sepal_width   90 non-null     float64
 2   petal_length  90 non-null     float64
 3   petal_width   90 non-null     float64
 4   species       90 non-null     object 
dtypes: float64(4), object(1)
memory usage: 3.6+ KB


In [473]:
X_train = df[df.columns[:-1]]
print(X_train.head())
X = X_train.to_numpy(dtype="float32")
X.shape

   sepal_length  sepal_width  petal_length  petal_width
0           4.9          3.0           1.4          0.2
1           4.7          3.2           1.3          0.2
2           4.6          3.1           1.5          0.2
3           5.0          3.6           1.4          0.2
4           5.4          3.9           1.7          0.4


(90, 4)

In [474]:
number_features = len(X_train.columns)  
number_samples = len(X_train)
number_samples

90

In [475]:
number_features

4

In [476]:
outcome = df[df.columns[-1]]
outcome

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
           ...       
85    Iris-versicolor
86    Iris-versicolor
87    Iris-versicolor
88    Iris-versicolor
89    Iris-versicolor
Name: species, Length: 90, dtype: object

In [477]:
Y_train = outcome == "Iris-setosa"
Y_train = Y_train.astype("int")

In [478]:
Y_train[Y_train==0] = -1
Y = np.array(Y_train)

In [479]:
number_features = len(df.columns) - 1  # We don't take the species column
number_samples = len(df)
number_samples

90

In [480]:
number_features

4

# Solution
As seen in class the goal of this homework is to classify two types of flowers. To do so we have to find a hyperplane that divide the space in two regions, one for the _Iris-setosa_ and the other for the _Iris-versicolor_. To do so we have to sole the following optimization problem:

$$
\text{minimize} \;\;\; \|\omega\|_2
$$
$$
\text{s.t.}\;\;\; y_i(\omega^\top x_i+b)\geq 1,
$$

where $\omega$ is the normal vector to the hyperplane, $b$ is the coordinate at the origin, $x_i$ is a vector in $\mathbb{R}^4$ containing the characteristics of one flower and $y_i$ is either 1 or -1, depending on the type of flower. The problem is completely equivalent to 

$$
\text{minimize} \;\;\; \frac{1}{2}\|\omega\|_2^2
$$
$$
\text{s.t.}\;\;\; 1-y_i(\omega^\top x_i+b)\leq 0,
$$

because $\|\omega\|_2$ and $\|\omega\|^2_2$ share the point where they achieve the minimum. This is a problem that we know how to solve because using a barrier method we can convert it into an unconstrained problem. We are going to use the logarithmic barrier and the problem becomes

$$
\text{minimize} \;\;\; f(\omega,b) = \frac{1}{2}\|\omega\|_2^2 - \left(\frac{1}{t}\right)\sum_{i=1}^n \log\left[y_i(\omega^\top x_i+b) -1\right],
$$

where $n$ is the number of samples, in our case 90. Now our problem with inequality constraints has become one without any constraint, and we have to minimize this function with respect to the parameters $\omega\in\mathbb{R}^4$ and $b\in\mathbb{R}$. To do so we are going to use the Newton method, so we need to compute the gradient and the Hessian of our function. In our case the gradient is $\nabla\to\left(\nabla_\omega,\partial_b\right)$

$$
\nabla_\omega f(\omega,b) = \omega - \left(\frac{1}{t}\right)\sum_{i=1}^n \frac{y_ix_i}{y_i(\omega^\top x_i+b) -1}
$$
$$
\partial_b f(\omega,b) = - \left(\frac{1}{t}\right)\sum_{i=1}^n \frac{y_i}{y_i(\omega^\top x_i+b) -1}. 
$$
As we can see $\nabla_\omega f(\omega,b)$ is a vector because of the factor $x_i$ in the numerator, and $\partial_b f(\omega,b)$ is a scalar because $y_i$ is a scalar.



# Code

In [481]:
def f(point, mu, y_vector, X, n = 90):
    """The function to minimize

    Args:
        point: is an array in which the first element is b
            and the rest are vector w.
        mu: the parameter that multiplies the logarithmic barrier,
            1/t in the formula.
        y_vector: is the array containing all the values of y_i.
        X: is the matrix with all the samples features
        n: the number of samples.
    """
    assert point.shape == (5,), "Incorrect size of point. The size must be 5"

    b = point[0]
    w = point[1:]

    # We create some matrices to compute the function f without explicit summations
    one = np.ones(shape=(n,))
    w_matrix = np.tile(w, (n, 1)).T  # Is a matrix whose columns are the vector w
    b_vector = np.full_like(y_vector, fill_value=b)

    #                                     this is equivalent to np.diag(W.T.dot(X))
    return 0.5*w.T@w - mu*one.T@np.log(y_vector * ((w_matrix * X).sum(0) + b_vector) - one)

def gradient(point, mu, y_vector, X, n = 90):
    """The gradient of the function to minimize

    Args:
        point: is an array in which the first element is b
            and the rest are vector w.
        mu: the parameter that multiplies the logarithmic barrier,
            1/t in the formula.
        y_vector: is the array containing all the values of y_i.
        X: is the matrix with all the samples features
        n: the number of samples.
    """
    b = point[0]
    w = point[1:]

    # We create some matrices to compute the function f without explicit summations
    one = np.ones(shape=(n,))
    w_matrix = np.tile(w, (n, 1)).T  # Is a matrix whose columns are the vector w
    b_vector = np.full_like(y_vector, fill_value=b)
    denom = y_vector * ((w_matrix * X).sum(0) + b_vector) - one

    grad_w = w - mu * np.sum((Y*X) / denom, axis=1)
    grad_b = -mu * np.sum(Y/denom)
    return np.concatenate((grad_w, grad_b), axis=None)

def hessian(point, mu, y_vector, X, n = 90):
    b = point[0]
    w = point[1:]

    # We create some matrices to compute the function f without explicit summations
    one = np.ones(shape=(n,))
    w_matrix = np.tile(w, (n, 1)).T  # Is a matrix whose columns are the vector w
    b_vector = np.full_like(y_vector, fill_value=b)
    denom = y_vector * ((w_matrix * X).sum(0) + b_vector) - one

    # First part of the hessian of the barrier
    full_grad = np.vstack((Y*X, Y)) # Each column is (\nabla f_i)
    total = np.einsum('ij,ik->ijk', full_grad.T, full_grad.T)/denom[:, None, None]**2
    hess_1 = total.sum(axis=0)

    # Second part of the Hessian of the barrier is 0

    return np.identity(hess_1.shape[0]) - mu*hess_1

def line_search(d_k, x, mu, y_vector, X):
    """Implement the line search method, this function update the time step
    until the criterion is fulfilled"""

    alpha = 0.35
    beta = 0.5
    t = 1
    while f(x + t*d_k, mu, y_vector, X) > f(x, mu, y_vector, X) + alpha * t * np.dot(gradient(x, mu, y_vector, X),d_k):
        t *= beta
    return t



In [482]:
def newtons_method(stop, x, mu, y_vector, X):
    """Implementation of the newton method"""

    grad = gradient(x, mu, y_vector, X)
    if np.dot(grad, grad) < stop**2:  # check the case that we are already in a minimum
        return x, 0

    for _ in range(1,1000):
        descend_direction = npl.solve(-hessian(x, mu, y_vector, X), grad)
        t = line_search(descend_direction, x, mu, y_vector, X)
        x = x + t * descend_direction
        grad = gradient(x, mu, y_vector, X)
        if np.dot(grad, grad) < stop**2:
            return x

    print("The stop criterium wasn't achieve in 10000 iterations.")
    return x


To get our starting feasible point we choose two elements of the data set, one for the Iris-setosa and another from the Iris-versicolor. The easiest way of doing this is to take the first element $p$ and last element $q$ of the data set. As a starting vector we take the vector from one point to the other, $p-q$ and as a starting $b$ whe take the average of the components

In [483]:
p = X[0]
q = X[-1]

w_0 = p - q
b = np.mean(p+q)
point_0 = np.append(b, w_0)

In [484]:
newtons_method(10e-5, point_0, 0.1, Y, X.T)

C:\Users\arcos\AppData\Local\Temp\ipykernel_11544\1706675037.py:24: RuntimeWarning: invalid value encountered in log
  return 0.5*w.T@w - mu*one.T@np.log(y_vector * ((w_matrix * X).sum(0) + b_vector) - one)


The stop criterium wasn't achieve in 10000 iterations.


(array([ 8.67727996e+13, -3.58667945e+11,  1.11074393e+09, -2.29084518e+06,
         2.35993321e+03]),
 999)

In [485]:
def fit(point_0, Y, X):
    """We start with mu big enough so our function is differentiable 
    and we decrease the value of mu until a certain point
    """
    mu = 1
    point = newtons_method(10e-5, point_0, mu, Y, X.T)
    pass

In [486]:
# X = X.T
# X_2 = X.T
# X_2.shape

In [487]:
# X_ampli_2 = np.vstack((Y*X_2, Y))
# B_2 = np.einsum('ij,ik->ijk', X_ampli_2.T, X_ampli_2.T)
# sumnor = B_2.sum(axis=0)
# sumnor.shape

In [488]:
# print(X_ampli_2)
# print()
# print(X_ampli_2/np.full_like(Y,fill_value=2.0))